In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [11]:
class PINN(nn.Module):
    def __init__(
            self, 
            num_inputs: int=1,
            num_hidden_layers: int=1,
            num_neurons: int=1,
            num_outputs: int=1,
            activation: nn.Module = nn.Tanh()
    ):
        super().__init__()
        self.num_inputs = num_inputs
        self.num_hidden_layers = num_hidden_layers
        self.num_neurons = num_neurons
        self.num_outputs = num_outputs
        self.activation = activation

        layers = [nn.Linear(num_inputs, num_neurons)]

        for _ in range(num_hidden_layers):
            layers.append(activation)
            layers.append(nn.Linear(num_neurons, num_neurons))
        
        layers.append(activation)
        layers.append(nn.Linear(num_neurons, num_outputs))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [12]:
R = 1.0
X_BOUNDARY=0.0
FUN_BOUNDARY=0.0
NUM_INPUTS = 1
NUM_OUTPUTS = 1
DOMAIN = (-0.9, 0.9)

In [13]:

def loss_function(
        model: nn.Module, 
        x: torch.Tensor,
        bx: torch.Tensor = torch.Tensor([X_BOUNDARY]), 
        bu: torch.Tensor = torch.Tensor([FUN_BOUNDARY]),
    ) -> torch.Tensor:
    u = model(x)
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]

    pde = u_x - 1/(1 - torch.square(x)) 
    bc = model(bx) - bu

    loss = torch.mean(pde**2) + torch.mean(bc**2)

    return loss

In [14]:
NUM_LAYERS = 10
NUM_NEURONS = 10
EPOCHS = 100000 #how many times the training step is performed
BATCH_SIZE = 20 # how many input values are considered for each epoch
LEARNING_RATE = 5e-5
TOLERANCE = 1e-5

In [15]:
from IPython.display import clear_output

In [16]:
model = PINN(num_inputs=NUM_INPUTS,  num_hidden_layers= NUM_LAYERS, num_neurons=NUM_NEURONS, num_outputs=1)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [17]:
def train_step(model,input):
    optimizer.zero_grad()
    loss = loss_function(model, input)
    loss.backward()
    optimizer.step()

    return loss


In [18]:
for epoch in range(EPOCHS):
    input_batch =  torch.rand(BATCH_SIZE, NUM_INPUTS) * (DOMAIN[1] - DOMAIN[0]) + DOMAIN[0] 
    input_batch.requires_grad_(True)
    loss = train_step(model, input_batch) 
    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Loss: {loss}')
    if loss <= TOLERANCE:
        break

clear_output()
print(f'Terminated at epoch {epoch} with loss {loss}')

Epoch: 0, Loss: 2.34452223777771
Epoch: 1000, Loss: 4.498660564422607
Epoch: 2000, Loss: 0.8621701002120972
Epoch: 3000, Loss: 0.27699851989746094
Epoch: 4000, Loss: 0.1740504801273346
Epoch: 5000, Loss: 0.4873426556587219
Epoch: 6000, Loss: 0.6641388535499573
Epoch: 7000, Loss: 0.20552760362625122
Epoch: 8000, Loss: 0.12256646901369095
Epoch: 9000, Loss: 0.25120866298675537


In [12]:
def analytical_sol_fn(
        x: torch.Tensor
)-> torch.Tensor:
    return (-1/2)*torch.log(1-x) + (1/2)*torch.log(1+x)

x_plot = torch.linspace(DOMAIN[0], DOMAIN[1], 1000).reshape(-1, 1)
y_pred = model(x_plot)
y_anlt = analytical_sol_fn(x_plot)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x_plot.detach(), y_pred.detach(), label='Predictions')
plt.plot(x_plot.detach(), y_anlt.detach(), label='analytical', linestyle='dashed')
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Neural Network Predictions vs. Analytical Solution')
plt.legend()
plt.grid(True)
plt.show()